# Cleaning the data and merge into one dataframe

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>")) #reduce margins in jupyter notebook

In [ ]:
#todo clean superlatives (longest outliers, replace 0's in columns that contain mostly 0's with NaN), other.

In [21]:
import pandas as pd
# from fuzzywuzzy import fuzz # improve our match numbers so that we have more data to run regression on

In [3]:
# def match_name(name, list_names, min_score=0): # code from https://medium.com/@rtjeannier/combining-data-sets-with-fuzzy-matching-17efcb510ab2
#     # -1 score incase we don't get any matches
#     max_score = -1
#     # Returning empty name for no match as well
#     max_name = ""
#     # Iternating over all names in the other
#     for name2 in list_names:
#         #Finding fuzzy match score
#         score = fuzz.ratio(name, name2)
#         # Checking if we are above our threshold and have a better score
#         if (score > min_score) & (score > max_score):
#             max_name = name2
#             max_score = score
#     return (max_name, max_score)

In [4]:
million_songs = pd.DataFrame.from_csv('/Users/horisustar/projects/project2backup/million_songs.csv',sep=',') #import csvs from other notebooks
top_100 = pd.DataFrame.from_csv('/Users/horisustar/projects/project2backup/historical_top_100.csv',sep=',') # check our csv

/Users/horisustar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.
/Users/horisustar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


In [5]:
top_100.head() # which columns should we merge on?

,Artist,Rating,Song,Year
0,Percy Faith,1,Theme From “A Summer Place”,1960
1,Jim Reeves,2,He’ll Have To Go,1960
2,Everly Brothers,3,Cathy’s Clown,1960
3,Johnny Preston,4,Running Bear,1960
4,Mark Dinning,5,Teen Angel,1960


In [8]:
top_100.rename(columns = {'Artist': 'artist_name','Song':'title','Year':'year','Rating':'rating'}, inplace = True) # rename to match columns between dbs
top_100.columns

Index(['artist_name', 'rating', 'title', 'year'], dtype='object')

In [9]:
million_songs.columns #take a look at the column names to see what to drop

Index(['track_id', 'title', 'song_id', 'release', 'artist_id', 'artist_mbid',
       'artist_name', 'duration', 'artist_familiarity', 'artist_hotttnesss',
       'year', 'danceability', 'energy', 'key', 'key_confidence', 'loudness',
       'mode', 'mode_confidence', 'song_hotttnesss', 'tempo', 'time_signature',
       'time_signature_confidence', 'track_7digitalid', 'shs_perf',
       'shs_work'],
      dtype='object')

In [10]:
drop_list = ['year','artist_mbid','shs_perf','shs_work', 'track_7digitalid', 'artist_id', 'song_id',\
             'track_id', 'shs_perf', 'shs_work', 'danceability', 'energy', 'key_confidence',\
             'time_signature_confidence', 'mode_confidence'] # irrelevant or broken columns from million_songs

In [11]:
million_songs.drop(drop_list,axis=1,inplace=True) # drop the year col so we can match on artist name and title only 
                                                  # without duplicate year cols
million_songs.columns

Index(['title', 'release', 'artist_name', 'duration', 'artist_familiarity',
       'artist_hotttnesss', 'key', 'loudness', 'mode', 'song_hotttnesss',
       'tempo', 'time_signature'],
      dtype='object')

In [ ]:
million_songs = million_songs[million_songs.Song != 0]

In [12]:
million_songs.shape

(838276, 12)

In [13]:
top_100.shape

(5300, 4)

### With clean column names and the features we want selected from the larger db, we can perform our merge, but we need to make sure we match weird song titles - TODO

In [22]:
# # code reference from https://medium.com/@rtjeannier/combining-data-sets-with-fuzzy-matching-17efcb510ab2

# # List for dicts for easy dataframe creation 
# dict_list = []
# # iterating over our players without salaries found above
# for name in missing_salaries.player_name:
#     # Use our method to find best match, we can set a threshold here
#     match = match_name(name, df_salaries.player_name, 75)
    
#     # New dict for storing data
#     dict_ = {}
#     dict_.update({"player_name" : name})
#     dict_.update({"match_name" : match[0]})
#     dict_.update({"score" : match[1]})
#     dict_list.append(dict_)
    
# merge_table = pd.DataFrame(dict_list)
# # Display results
# merge_table

In [15]:
match_cols = ['artist_name', 'title'] # columns we want to merge on

In [16]:
ranked_songs = top_100.merge(million_songs,on=match_cols) # adding rank and ranking year to larger db

In [17]:
ranked_songs.shape

(1450, 14)

In [128]:
ranked_songs.head(20)

,artist_name,rating,title,year,release,duration,artist_familiarity,artist_hotttnesss,key,loudness,mode,song_hotttnesss,tempo,time_signature
0,Mark Dinning,5,Teen Angel,1960,True Rock N Roll 3CD Set,158.95465,0.469933,0.334437,0,-12.697,1,-1.000000,100.699,4
1,Jimmy Jones,8,Handy Man,1960,The Greatest Rock 'n' Roll Hits,123.89832,0.550149,0.346467,10,-12.036,1,-1.000000,143.655,4
2,Chubby Checker,10,The Twist,1960,Hits of The 60's Volume 1,164.36200,0.631271,0.439454,11,-9.814,0,-1.000000,154.208,7
3,Chubby Checker,9,The Twist,1962,Hits of The 60's Volume 1,164.36200,0.631271,0.439454,11,-9.814,0,-1.000000,154.208,7
4,Bobby Rydell,12,Wild One,1960,The Ultimate Sixties Collection,136.88118,0.504600,0.368641,7,-8.317,1,0.731105,151.581,3
5,Marty Robbins,15,El Paso,1960,Backline Vol. 66,260.44036,0.609897,0.479046,2,-10.957,1,-1.000000,105.657,3
6,Hollywood Argyles,16,Alley-oop,1960,Crusin' The 66 Vol. 2,168.20200,0.377071,0.012653,2,-11.034,0,0.611545,129.096,4
7,Frankie Avalon,24,Why,1960,Frankie Avalon,157.72689,0.574053,0.395116,5,-16.817,1,0.000000,93.523,1
8,Drifters,26,Save The Last Dance For Me,1960,Magic Moments,136.59383,0.618409,0.383136,2,-10.472,1,-1.000000,160.921,4
9,Jimmy Clanton,33,Go Jimmy Go,1960,Crusin' The 66 Vol. 2,138.70975,0.496255,0.382346,4,-16.669,0,-1.000000,133.213,4


In [18]:
ranked_songs.to_csv('final_ranked_song_db.csv') # export our csv

In [19]:
csv_check = pd.DataFrame.from_csv('/Users/horisustar/projects/project2backup/final_ranked_song_db.csv',sep=',') # check our csv

/Users/horisustar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [20]:
print(csv_check) # all good!

                          artist_name  rating                          title  \
0                        Mark Dinning       5                     Teen Angel   
1                         Jimmy Jones       8                      Handy Man   
2                      Chubby Checker      10                      The Twist   
3                      Chubby Checker       9                      The Twist   
4                        Bobby Rydell      12                       Wild One   
5                       Marty Robbins      15                        El Paso   
6                   Hollywood Argyles      16                      Alley-oop   
7                      Frankie Avalon      24                            Why   
8                            Drifters      26     Save The Last Dance For Me   
9                       Jimmy Clanton      33                    Go Jimmy Go   
10                        Toni Fisher      36                   The Big Hurt   
11                      Conway Twitty   